In [1]:
import pandas as pd
import numpy as np

class SME():
    
    def __init__(self):
        self.asked = 0
        self.df = self.get()
        
    def get(self):
        X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')
        y_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_y.csv')

        X_train['Left'] = y_train
        return X_train
    
    def ask(self, valuedict):
        self.asked += 1
        #  print(self.asked)
        
        if self.asked > 500000:
            raise Exception("Sorry, you have asked enough")

        arr = []
        for prop in valuedict:
            val = valuedict[prop]

            if val is None: 
                continue

            val = f"'{val}'" if isinstance(val,str) else val
            # print(prop, val)
            arr.append(f'{prop} == {val}')

            query = ' and '.join(arr)
        result = self.df.query(query)

        if len(result) == 0:
            raise Exception("I don't know")
        
        return (int)(result['Left'].mean())

In [2]:
sme = SME()

In [3]:
sme.df.columns

Index(['RecordId', 'Gender', 'Distance', 'YearsWorked', 'TrainingHours',
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'DepartmentCode', 'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Left'],
      dtype='object')

In [4]:
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')

In [5]:
sme.df['SelfReview'] = sme.df['SelfReview'].fillna(sme.df['SelfReview'].median())
sme.df['SupervisorReview'] = sme.df['SupervisorReview'].fillna(sme.df['SupervisorReview'].median())
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].astype(int)

sme.df['Raise'] = (sme.df['Salary'] / sme.df['PreviousSalary']) - 1
sme.df['ReviewDiff'] = sme.df['SelfReview'] - sme.df['SupervisorReview']
sme.df['ReviewOverRaise'] = sme.df['SelfReview'] / (sme.df['Raise'] +.0001)

sme.df['LongDist'] = np.where(sme.df['Distance'].isin(['~20miles', '>30miles']), 1, 0)

sme.df['HighStress'] = np.where(sme.df['StressLevel'] > 3.5, 1, 0)

num = ['HighStress', 'LongDist','Gender', 'SupervisorReview','SelfReview', 'Raise', 'Salary','ReviewOverRaise'] #'ReviewOverRaise','ReviewDiff', 'StressLevel'
cat = ['DepartmentCode']#'Distance',
#num = ['Raise', 'StressLevel']

"""
num = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', 
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']
"""

"\nnum = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', \n       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',\n       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',\n       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',\n       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',\n       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',\n       'PreviousSalary', 'Salary', 'SelfReview',\n       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']\n"

# Establish the preprocessing for numeric and categorical features.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

n_clusters = 500

featureprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num)#,
        #('cat', OneHotEncoder(), cat)
    ])
preprocessor = Pipeline(steps=[
    ('features', featureprocessor),
    ('pca', PCA(n_components=0.9)),
    ('kmeans', KMeans(n_clusters=n_clusters, random_state=42, n_init='auto'))
])
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto'), num)#+cat)
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto')

In [8]:
X_dist = preprocessor.fit_transform(sme.df[num+cat])

In [9]:
preprocessor

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['HighStress', 'LongDist',
                                                   'Gender', 'SupervisorReview',
                                                   'SelfReview', 'Raise',
                                                   'Salary',
                                                   'ReviewOverRaise'])])),
                ('pca', PCA(n_components=0.9)),
                ('kmeans',
                 KMeans(n_clusters=500, n_init='auto', random_state=42))])

In [10]:
kmeans = KMeans(n_clusters = n_clusters, n_init='auto')
pre = featureprocessor.fit_transform(sme.df[num+cat])

In [11]:
X_dist = kmeans.fit_transform(pre)
X_dist

array([[4.00171923, 3.3079579 , 3.72659918, ..., 2.56150844, 6.85420143,
        4.03004859],
       [2.78972543, 3.5378403 , 3.5091031 , ..., 3.45057363, 6.91032054,
        3.94326496],
       [2.6969939 , 5.39149026, 3.70605857, ..., 2.42249429, 5.98651671,
        4.40220978],
       ...,
       [3.92809928, 4.4136642 , 4.60394597, ..., 4.03977881, 6.99196109,
        3.81065053],
       [3.9916972 , 3.81180185, 2.76990713, ..., 3.97026524, 6.9060077 ,
        4.18793842],
       [4.10959185, 3.80422101, 3.60736346, ..., 2.57925644, 7.12992874,
        3.04853284]])

In [12]:
X_dist.shape

(500000, 500)

In [13]:
pd.Series(kmeans.labels_).value_counts()

26     3969
202    3464
10     3416
43     3195
311    2971
       ... 
191     222
441     213
377     207
392     186
264     176
Name: count, Length: 500, dtype: int64

In [14]:
import numpy as np
representative_idx = np.argmin(X_dist, axis=0)
representative_idx.shape

(500,)

In [15]:
representative_idx

array([ 17740,     65,   2713,   5232, 211042,  75331,   6118,   3228,
        12383,   1630,   1988,   6828,   1391,   6645,  47981,    529,
        15080,   6332,  18487,   4851, 172524,   2269,   3994,   1844,
         1783,   4704,   2123,   6407,   1376,  14323,   9481,   8027,
         2913,   2282,   6497,   1520,    292,   5438,  21554,   1699,
         2671,   1683,  24804,    377,   5705,   5524,  76311,    179,
         6979,   4610,   2917,   6241,    651,  97612, 113909,  15088,
        11716,   2789,   4650,   9935,    298,     27,   2399,    280,
         4203,    414,   6498,   1655,   2317,  89428,   1701, 181275,
         4424,  19382,   2780,  10237,  32627,   2003,    806,   6570,
         7448,   2149,   3143,   6666,   6133,   4064,     41,    165,
         4711,   3072,   4057,   8377,    725,  65948,   2273,   2519,
          763,  13118,    512,  85291,   5021,  29620,   7263,   2237,
           20,   8272,   7647,   2183,   9309,   2018,  12496,   2138,
      

In [16]:
X_representative = sme.df.iloc[representative_idx]
X_representative

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,PreviousSalary,Salary,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress
17740,17840,0,~15miles,3,62,1,16,5,8,1,...,58000,60000,3.0,3.0,1,0.034483,0.0,86.748430,0,0
65,165,1,~20miles,7,48,4,17,4,14,0,...,57000,59000,5.0,5.0,0,0.035088,0.0,142.095029,1,1
2713,2813,0,~15miles,5,92,4,2,4,15,0,...,87000,89000,5.0,5.0,0,0.022989,0.0,216.557973,0,0
5232,5332,0,~15miles,5,50,4,2,2,15,1,...,67000,70000,4.0,4.0,0,0.044776,0.0,89.134267,0,0
211042,211142,0,~20miles,6,78,4,2,2,13,0,...,55000,55000,5.0,4.0,1,0.000000,1.0,50000.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265698,265798,1,~20miles,7,74,2,11,2,18,1,...,83000,86000,4.0,2.0,0,0.036145,2.0,110.361334,1,1
28388,28488,0,>30miles,9,21,4,10,2,13,1,...,81000,83000,5.0,2.0,1,0.024691,3.0,201.683183,1,0
46671,46771,1,~15miles,10,79,3,6,4,6,0,...,80000,82000,4.0,3.0,0,0.025000,1.0,159.362550,0,0
7011,7111,0,~15miles,7,21,4,14,3,14,1,...,81000,81000,5.0,2.0,1,0.000000,3.0,50000.000000,0,0


In [17]:
y_representative= []
for i in representative_idx:
    y_representative.append(sme.ask({'RecordId': i+100})) #All record Id's are 100 more than the index value

In [18]:
y_train_propagated = np.empty(len(sme.df), dtype=np.int32)
for i in range(n_clusters):
    y_train_propagated[kmeans.labels_ == i] = y_representative[i] #Issue is here

In [19]:
sme.df['Labels'] = kmeans.labels_
sme.df['y_pred'] = y_train_propagated

In [20]:
sme.df['ClusteringOff']= abs(sme.df['y_pred'] - sme.df['Left'])

In [78]:
#What Percent incorrect is each cluster?
percent_df = sme.df.groupby('Labels')['ClusteringOff'].sum()/sme.df.groupby('Labels')['ClusteringOff'].count()
percent_df[percent_df>.5].count()

84

In [22]:
sme.df['ClusteringOff'].sum() / sme.df['ClusteringOff'].count()

0.246576

In [52]:
#What percetange of "Leaving" predictions did our model get wrong?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum() / sme.df[sme.df['Left']==1]['ClusteringOff'].count()

0.3344877840641468

In [54]:
#What percetange of "Staying" predictions did our model get wrong?
sme.df[sme.df['Left']==0]['ClusteringOff'].sum() / sme.df[sme.df['Left']==0]['ClusteringOff'].count()

0.1969274486056475

In [70]:
sme.df[sme.df['Left']==1]['ClusteringOff'].sum()

60362

In [68]:
sme.df[sme.df['Left']==0]['ClusteringOff'].sum()

62926